In [ ]:
# # GoogleDirve の指定したディレクトで作業する場合
# %cd /content/drive/My Drive/project_directory

In [ ]:
# setting
youtube_url = "https://youtu.be/QcpQ7rqT5fA"
filename = "test_1"
is_save = True

# step1 : clone repositories

In [ ]:
!git clone "https://github.com/TakanariShimbo/bgst.git"

In [ ]:
!git clone "https://github.com/TakanariShimbo/yolov7.git"

In [ ]:
!git clone "https://github.com/TakanariShimbo/balldet.git"

# step2 : prepare movie

In [ ]:
import os

In [ ]:
# make dirs
os.makedirs("datas/original_movies", exist_ok=True)

In [ ]:
# insltall youtube dl
!pip install youtube-dl

In [ ]:
# check format code
!youtube-dl -F {youtube_url}

In [ ]:
# download movie
save_path = f"datas/original_movies/{filename}.mp4"
!youtube-dl -o {save_path} -f 134 {youtube_url}

# step3 : detect ball

In [ ]:
import time
import os
import shutil
import subprocess

In [ ]:
def bgst():
    total_frame = 1001
    down_fps_rate = 1
    down_resolution_rate = 1.0
    
    # ---- bgst ----
    time.sleep(1)
    os.chdir("bgst")

    # flow1. convert movie to images
    print("flow1. convert movie to images")
    # run
    !python convert_movie2images.py --filename {filename} --total_frame {total_frame} --down_fps_rate {down_fps_rate} --down_resolution_rate {down_resolution_rate}

    # flow2. make mask images
    print("flow2. make mask images")
    # run
    !python make_mask.py --filename {filename}

    time.sleep(1)
    os.chdir("../")

In [ ]:
def yolo():
    weights = "models_checkpoint/yolov7.pt"
    source = f"../datas/original_images/{filename}"
    project = "../datas/humans_bbox"
    
    # --- yolo ----
    time.sleep(1)
    os.chdir("yolov7")

    # flow3. detect humans bbox
    print('flow3. detect humans bbox')
    # deleate old folder
    save_path = f"../datas/humans_bbox/{filename}"
    if os.path.exists(save_path):
        print('yolo, delete previous folder.')
        shutil.rmtree(save_path)
    else:
        print("yolo, previous folder doesn't exist.")
    # run
    !python detect.py --save-txt --device 0 --classes 0 --weights {weights} --source {source} --project {project} --name {filename}

    time.sleep(1)
    os.chdir("../")

In [ ]:
def balldet():
    original_fps = 30
    slow_rate = 4
    
    # ---- balldet ----
    time.sleep(1)
    os.chdir("balldet")

    # flow4. make mask2 images
    print("flow4. make mask2 images")
    # run
    !python make_mask2.py --filename {filename}

    # flow5. detect ball
    print("flow5. detect ball")
    # run
    !python detect_ball.py --filename {filename}

    # flow6. convert balldet images to movie
    print("flow6. convert balldet images to movie")    
    # run
    !python convert_images2movie.py --filename {filename} --original_fps {original_fps} --slow_rate {slow_rate}

    time.sleep(1)
    os.chdir("../")

In [ ]:
bgst()

In [ ]:
yolo()

In [ ]:
balldet()

In [ ]:
if is_save:
    from google.colab import files
    files.download(f"datas/balldet_movies/{filename}.mp4") 